# 列表和数组

高性能编程最重要的事情是了解你的数据结构所能提供的性能保证。(understanding the guarantees of the data structures you use.)

所以，我们需要掌握：
- understanding what questions you are trying to ask of your data and 
- picking a data structure that can answer these questions quickly

#### 编写高效代码的原则：
- 选择正确的数据结构并坚持使用它
    - 注意，有时为了使用更高效的数据结构，可能需要更高的代价来构建它(例如，字典)。
- 通用的代码会比为某个特定问题设计的代码慢得多
    - 例如，列表的数据类型可能不同，如果都为数字，使用numpy 会更高效。
    - 非数字的数据，可以使用blist 和array



## 1. 数组 (Array)
Lists and tuples are a class of data structures called arrays.

通常，我们创建一个数组的时候，需要和操作系统kernel 交互，在内存中分配M 个连续的地址。例如下图所示，我们在内存中分配了6个连续的地址空间，第一个元素是列表的长度，后面五个地址空间用于存储对象的地址引用。

<img src="figures/arrays.png" alt="drawing" width="600"/>

通过index 对list 的查找时间是$O(1)$. 下面我们可以看出，两个长度不同的lists，获取其特定index 的元素时间相同(极为相近)。

In [4]:
l = range(10)
print('lenth of list: ', len(l))
%timeit l[5]

lenth of list:  10
99.1 ns ± 1.81 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [5]:
l = range(1000000)
print('lenth of list: ', len(l))
%timeit l[5]

lenth of list:  1000000
96.1 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


如果我们想通过值来获取它在一个列表中的index(例如，判断某list 中是否包含某元素)，解决这个方法的基本方法叫线性搜索, 其算法复杂度为$O(n)$。

In [10]:
# list.index() 的方法

def linear_search(value, lst):
    for i, item in enumerate(lst):
        if item == value:
            return i
    return -1

In [9]:
lst = [9, 18, 18, 19, 29, 42, 56, 61, 88, 95]

print(linear_search(18, lst))
print(linear_search(1, lst))

1
-1


高效搜索的必备要素：
- 排序算法
- 搜索算法

解决这个问题的一个方法是，首先对列表进行排序，然后使用较快的二分查找。

### 1.1 排序

Python内建的排序使用了Tim Sort, 混用了插入排序 (insertion sort)和合并排序(merge sort). 在最差情况下复杂度为$O(n)$.

### 1.2 搜索

二分搜索复杂度$O(log n)$

In [11]:
def binary_search(value, lst):
    imin, imax = 0, len(lst)
    while True:
        if imin >= imax:
            return -1
        
        mid_point = (imin + imax) // 2
        
        if lst[mid_point] > value:
            imax = mid_point
            
        elif lst[mid_point] < value:
            imin = mid_point
            
        else: 
            return mid_point

In [12]:
print(binary_search(18, lst))
print(binary_search(1, lst))

2
-1


我们可以看出，我们找到了第二个18，而非第一次出现的index.

#### list v.s. dict

- 查找字典的时间复杂度为$O(1)$, 但是构建字典的复杂读为$O(n)$.
- 字典要求没有重复的key，list 中可以用相同的元素

#### bisect module

- `bisect.insort(lst, value)` 保持排序的同时往列表中添加元素
- `bisect.bisect_left(lst, value)` 基于优化的二分查找，返回离搜索值最近的元素(并大于搜索值)的index

In [26]:
import bisect
import random

important_numbers = []
for i in range(10):
    new_number = random.randint(0, 1000)
    bisect.insort(important_numbers, new_number)

In [34]:
# important_numbers will already be in order because we inserted new elements
# with bisect.insort
print(important_numbers)

[161, 234, 275, 298, 312, 363, 640, 668, 744, 746]


In [ ]:
def find_closest(lst, value):
    # bisect.bisect_left will return the first value in the lst
    # that is greater than the value
    i = bisect.bisect_left(lst, value)
    
    print('i: ', i)
    
    if i == len(lst):
        return i - 1
    
    elif lst[i] == value:
        return i
    
    elif i > 0:
        j = i - 1
        
        # since we know the value is larger than needle (and vice versa for the
        # value at j), we don't need to use absolute values here
        if lst[i] - value > value - lst[j]:
            return j
    return i

In [41]:
closest_index = find_closest(important_numbers, -250)
print('closest_index: ', closest_index)
print("Closest value to -250: ", important_numbers[closest_index])

i:  0
closest_index:  0
Closest value to -250:  161


In [42]:
closest_index = find_closest(important_numbers, 500)
print('closest_index: ', closest_index)
print("Closest value to 500: ", important_numbers[closest_index])

i:  6
closest_index:  5
Closest value to 500:  363


In [43]:
closest_index = find_closest(important_numbers, 1100)
print('closest_index: ', closest_index)
print("Closest value to 1100: ", important_numbers[closest_index])

i:  10
closest_index:  9
Closest value to 1100:  746


## 2. 列表

#### 动态改变长度

当列表(长度为N)的空间用完，再执行append 操作时，Python会做如下操作：
- 创建一个新的列表，长度为M+N 
- 将N个元素复制到新的列表

因为copy 是一个很昂贵的操作(尤其当列表的长度很长的时候)，所以每次扩容，会在N 的基础上向后多扩M个元素，避免以后再进行append 操作时，需要再次移动前面的所有元素。

新分配的单元个数M 和N 的关系如下(此处需要check 以下)：

In [17]:
def calculate_M(N):
    return (N >> 3) + (3 if N < 9 else 6)

In [18]:
for N in range(10):
    print('M = ', calculate_M(N))

M =  3
M =  3
M =  3
M =  3
M =  3
M =  3
M =  3
M =  3
M =  4
M =  7


下图展示了一个列表在多次添加时的内存变化示例。

<img src="figures/list_append.png" alt="drawing" width="200"/>

第一次扩容(超额分配)发生在第一次向列表添加元素的时候。在列表创建是，完全是按需的。

## 3. 元组

元组和列表的区别总结与下表：

|               list               |                tuple               |
|--------------------------------|----------------------------------|
| 动态，长度可变，元素可变         | 静态，长度不可变，元素不可变       |
| 需要操作系统内核在内存中分配资源 | 资源缓存，无需操作系统内核分配资源 |
| 分配的资源一般大于实际所需资源   | 分配资源等于实际数据所需资源       |
| 相对较重量级数据结构(比字典轻)   | 轻量级数据结构                     |

从下面我们可以看出，初始化一个数组要比初始化一个元组慢5倍左右。

In [19]:
%timeit l = [0,1,2,3,4,5,6,7,8,9]

90.8 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [20]:
%timeit l = (0,1,2,3,4,5,6,7,8,9)

16.9 ns ± 0.216 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
